<a href="https://colab.research.google.com/github/saugatabose28/work/blob/main/ModuleTesting_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==3.0.0
!pip install emoji
!pip install onnx onnxruntime
!pip install scikit-plot
!pip install plot-metric
!pip install pyod
!pip install transformers[onnx]
import tensorflow as tf
import gc

import os
os.environ['PYTHONHASHSEED']=str(1)

import random
import emoji as emoji
import re
import string
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
%matplotlib inline
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from numpy import dstack
from numpy import mean
from numpy import std
from numpy.linalg import norm
from transformers import AutoModel
from transformers import BertModel, BertTokenizer
# optimizer from hugging face transformers
from transformers import AdamW
from pyod.models.ocsvm import OCSVM
from pyod.utils.example import visualize#generate dataset
data_path="drive/My Drive/Colab Notebooks"
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
class BERT_Arch(nn.Module):
    
    def __init__(self, bert):
        super(BERT_Arch, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')        

    def forward(self, ids, mask):
          sequence_output, pooled_output = self.bert(ids,attention_mask=mask)
          return sequence_output

In [74]:
def read_dataset():
    data = pd.read_csv(f"{data_path}/preProcessedBalancedSampleDataset.csv")
    return data['tweet'].tolist(), data['class']

def data_process(data, labels):
    input_ids = []
    attention_masks = []
    bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    for sentence in data:
        bert_inp = bert_tokenizer.__call__(sentence, max_length=10,
                                           padding='max_length', pad_to_max_length=True,
                                           truncation=True, return_token_type_ids=False)

        input_ids.append(bert_inp['input_ids'])
        attention_masks.append(bert_inp['attention_mask'])

    input_ids = np.asarray(input_ids)
    attention_masks = np.array(attention_masks)
    labels = np.array(labels)
    d=np.array(data)
    return input_ids, attention_masks, labels,d

def load_and_process():
    data, labels = read_dataset()
    num_of_labels = len(labels.unique())
    #input_ids, attention_masks, labels = data_process(pre_process_dataset(data), labels)
    input_ids, attention_masks, labels,d = data_process(data, labels)

    return input_ids, attention_masks, labels,d
# function to train the model
def train():
    model.train()

    total_loss, total_accuracy = 0, 0

    # iterate over batches
    total = len(train_dataloader)
    
    correct=0
    print(total)
    seqOutput= torch.zeros(0)
    for i, batch in enumerate(train_dataloader):
        #print(batch)
        step = i+1
        percent = "{0:.2f}".format(100 * (step / float(total)))
        lossp = "{0:.2f}".format(total_loss/(total*batch_size))
        filledLength = int(100 * step // total)
        bar = '█' * filledLength + '>'  *(filledLength < 100) + '.' * (99 - filledLength)
        print(f'\rBatch {step}/{total} |{bar}| {percent}% complete, loss={lossp}, accuracy={total_accuracy}', end='')

        # push the batch to gpu
        batch = [b.to(device) for b in batch]
        sent_id, mask, labels = batch

        del batch
        gc.collect()
        torch.cuda.empty_cache()
        # clear previously calculated gradients
        model.zero_grad()

        output= model(sent_id, mask)
        #print(output[:, 0, :].size())
        #print(output.data)
        seqOutput=torch.cat((seqOutput, output[:, 0, :]),0)
        #seqOutput.append(output[:, 0, :])
        
        
        #print(output[:, 0, :].size(), labels.size())       
        loss = lossFunction(output[:, 0, :],labels)              
        #########Custom loss
        #loss = OCSVM_loss(weights,biases,output)
        #print(tf.math.reduce_mean(loss.item()))
        print(" training loss %f" %loss.item())
        #####################

        # add on to the total loss
        total_loss += float(loss.item())              
        
        # backward pass to calculate the gradients
        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        #update parameters
        optimizer.step()
        
    gc.collect()
    torch.cuda.empty_cache()

    # compute the training loss of the epoch
    avg_loss = total_loss / (len(train_dataloader)*batch_size)

    accu=100.*correct/total
    
    # returns the loss and predictions
    return avg_loss,seqOutput

# Specify the GPU
# Setting up the device for GPU usage
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Load Data-set ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
input_ids, attention_masks, labels,text = load_and_process()
df = pd.DataFrame(list(zip(input_ids, attention_masks)), columns=['input_ids', 'attention_masks'])

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ class distribution ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

# class = class label for majority of CF users. 0 - hate speech 1 - offensive language 2 - neither
# ~~~~~~~~~~ Split train data-set into train, validation and test sets  ~~~~~~~~~~#
#train_text, temp_text, train_labels, temp_labels = train_test_split(df, labels,
#                            random_state=2018, test_size=0.2, stratify=labels)

#val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels,
#                         random_state=2018, test_size=0.2, stratify=temp_labels)

#text,temp, labels,tempLabels = train_test_split(df, labels, test_size=0.0)

#train_text, val_text, train_labels, val_labels = train_test_split(temp_text, temp_labels,#
                         #random_state=2018, test_size=0.2, stratify=temp_labels)

#with open(data_path, )
a=[]
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
#for ids in test_text["input_ids"]:
#  a.append(tokenizer.decode(ids))
#d=pd.DataFrame(a)
#d.to_csv(f"{data_path}/testHasocBB.csv")


#del temp_text
gc.collect()
torch.cuda.empty_cache()

train_count = len(labels)
#test_count = len(test_labels)
#val_count = len(val_labels)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

# ~~~~~~~~~~~~~~~~~~~~~ Import BERT Model and BERT Tokenizer ~~~~~~~~~~~~~~~~~~~~~#
# import BERT-base pretrained model
bert = AutoModel.from_pretrained('bert-base-uncased')
# bert = AutoModel.from_pretrained('bert-base-uncased')
# Load the BERT tokenizer
#tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Tokenization ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# for train set
train_seq = torch.tensor(df['input_ids'].tolist())
train_mask = torch.tensor(df['attention_masks'].tolist())
train_y = torch.tensor(labels.tolist())

# for validation set
#val_seq = torch.tensor(val_text['input_ids'].tolist())
#val_mask = torch.tensor(val_text['attention_masks'].tolist())
#val_y = torch.tensor(val_labels.tolist())

# for test set
#test_seq = torch.tensor(test_text['input_ids'].tolist())
#test_mask = torch.tensor(test_text['attention_masks'].tolist())
#test_y = torch.tensor(test_labels.tolist())
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Create DataLoaders ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# define a batch size
batch_size=1
print(train_seq.size(),train_y.size())
# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)
print(len(train_data[0][0]))

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)
#print(len(train_sampler[0][0]))

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

#print(input_ids.shape)
#print(df.shape)
#print(train_data.get_shape())
#print((test_text))
#d=pd.DataFrame(test_text)
#d.to_csv(f"{data_path}/testDavid.csv")

# wrap tensors
#val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
#val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
#val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Freeze BERT Parameters ~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# freeze all the parameters
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)
# push the model to GPU
model = model.to(device)
#model=model1
for name, param in model.named_parameters():
  if name.startswith("bert"): # choose whatever you like here
    param.requires_grad = False

# define the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

#from sklearn.utils.class_weight import compute_class_weight

# compute the class weights
#class_wts = compute_class_weight('balanced', np.unique(train_labels), train_labels)

#print(class_wts)

# convert class weights to tensor
#weights = torch.tensor(class_wts, dtype=torch.float)
#weights = weights.to(device)

# loss function
#cross_entropy = nn.NLLLoss(weight=weights)

###try "squared hinge"###
#lossFunction=nn.MultiMarginLoss(p=2,reduction='sum')
###
###NLLL loss##
lossFunction = nn.NLLLoss()
###
###hinge loss##
#lossFunction=nn.HingeEmbeddingLoss();
####
###CE loss##
#lossFunction = nn.CrossEntropyLoss()
####

# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses = []
valid_losses = []

#if os.path.isfile("/content/drive/MyDrive/saved_weights.pth") == False:
#if os.path.isfile("saved_weights.pth") == False:
    # number of training epochs
epochs = 1
current = 1
# for each epoch
train_losses = []
valid_losses = []
#global co
while current <= epochs:

    print(f'\nEpoch {current} / {epochs}:')

    # train model
    
    train_loss,output= train()
    # evaluate model
    
    #valid_loss= evaluate()

    # save the best model
    #if valid_loss < best_valid_loss:
    #    best_valid_loss = valid_loss
        #torch.save(model.state_dict(), 'saved_weights.pth')

        # append training and validation loss
    train_losses.append(train_loss)
    #valid_losses.append(valid_loss)
    

    print(f'\n\nTraining Loss: {train_loss:.3f}')
    #print(f'Validation Loss: {valid_loss:.3f}')

    current = current + 1

cpu
torch.Size([64, 10]) torch.Size([64])
10

Epoch 1 / 1:
64
Batch 1/64 |█>..................................................................................................| 1.56% complete, loss=0.00, accuracy=0 training loss 0.559574


RuntimeError: ignored

In [73]:
print(output.size())

torch.Size([64, 768])
